# **Auto_report Generator**

In [2]:
pip install pandas fpdf


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40769 sha256=2b77759699f838dcd84676532b6cadaa6df48208aee53dc4f6b91c64e0682360
  Stored in directory: c:\users\e5400\appdata\local\pip\cache\wheels\aa\da\11\a3189f34ddc13c26a2d0f329eac46b728c7f31c39e4dc26243
Successfully built fpdf
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
from fpdf import FPDF
from datetime import datetime
import tkinter as tk
from tkinter import filedialog, messagebox

def generate_report():
    try:
        file_path = filedialog.askopenfilename(
            title="Select CSV File",
            filetypes=[("CSV Files", "*.csv")]
        )

        if not file_path:
            return

        df = pd.read_csv(file_path)

        # Required columns
        required_cols = ["product", "category", "units_sold", "unit_price", "discount"]
        missing = [col for col in required_cols if col not in df.columns]

        if missing:
            raise Exception(f"Missing Columns: {missing}")

        # Safe numeric conversion
        df["unit_price"] = pd.to_numeric(df["unit_price"], errors="coerce")
        df["units_sold"] = pd.to_numeric(df["units_sold"], errors="coerce")
        df["discount"] = pd.to_numeric(df["discount"], errors="coerce")

        df = df.dropna(subset=["unit_price", "units_sold", "discount"])

        # Calculations
        df["effective_price"] = df["unit_price"] * (1 - df["discount"] / 100)
        df["revenue"] = df["units_sold"] * df["effective_price"]

        total_revenue = df["revenue"].sum()
        total_units = df["units_sold"].sum()
        avg_order_value = df["revenue"].mean()

        top_product = df.groupby("product")["revenue"].sum().idxmax()
        top_category = df.groupby("category")["revenue"].sum().idxmax()

        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        pdf_name = f"Sales_Report_{timestamp}.pdf"

        # PDF
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", "B", 16)
        pdf.cell(0, 10, "Sales Summary Report", ln=True, align="C")

        pdf.set_font("Arial", size=12)
        pdf.ln(5)
        pdf.cell(0, 8, f"Generated On: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", ln=True)

        pdf.ln(5)
        pdf.set_font("Arial", "B", 13)
        pdf.cell(0, 8, "Summary Metrics:", ln=True)

        pdf.set_font("Arial", size=12)
        pdf.cell(0, 8, f"Total Revenue: PKR {total_revenue:,.0f}", ln=True)
        pdf.cell(0, 8, f"Total Units Sold: {total_units}", ln=True)
        pdf.cell(0, 8, f"Average Order Value: PKR {avg_order_value:,.0f}", ln=True)
        pdf.cell(0, 8, f"Top Product: {top_product}", ln=True)
        pdf.cell(0, 8, f"Top Category: {top_category}", ln=True)

        pdf.output(pdf_name)

        messagebox.showinfo("Success", f"Report generated successfully:\n{pdf_name}")

        # CLOSE APP AFTER SUCCESS
        app.destroy()

    except Exception as e:
        messagebox.showerror("Error", f"ERROR DETAILS:\n{str(e)}")

# GUI WINDOW
app = tk.Tk()
app.title("Auto Report Generator")
app.geometry("400x200")

label = tk.Label(app, text="Select a CSV file to generate the report:", font=("Arial", 12))
label.pack(pady=20)

btn = tk.Button(app, text="Choose CSV & Generate Report", font=("Arial", 12), command=generate_report)
btn.pack(pady=10)

app.mainloop()
